In [1]:
# read the api key
f = open("sec_api.key", "r")
api_key = f.readline()

In [3]:
import pandas as pd
# this is for us to toy with an already downloaded csv
df_10k = pd.read_csv('./spreadsheets/csv/10-K.csv')
print(df_10k['linkToFilingDetails'])

0     https://www.sec.gov/Archives/edgar/data/90168/...
1     https://www.sec.gov/Archives/edgar/data/196730...
2     https://www.sec.gov/Archives/edgar/data/178741...
3     https://www.sec.gov/Archives/edgar/data/12040/...
4     https://www.sec.gov/Archives/edgar/data/192958...
5     https://www.sec.gov/Archives/edgar/data/135968...
6     https://www.sec.gov/Archives/edgar/data/150625...
7     https://www.sec.gov/Archives/edgar/data/72633/...
8     https://www.sec.gov/Archives/edgar/data/196709...
9     https://www.sec.gov/Archives/edgar/data/932021...
10    https://www.sec.gov/Archives/edgar/data/110139...
11    https://www.sec.gov/Archives/edgar/data/115842...
12    https://www.sec.gov/Archives/edgar/data/143506...
13    https://www.sec.gov/Archives/edgar/data/189292...
14    https://www.sec.gov/Archives/edgar/data/197690...
15    https://www.sec.gov/Archives/edgar/data/783412...
16    https://www.sec.gov/Archives/edgar/data/808326...
17    https://www.sec.gov/Archives/edgar/data/13

In [3]:
urls = df_10k[['ticker', 
                'formType', 
                'periodOfReport',
                'filedAt', 
                'linkToFilingDetails']].rename(columns={'linkToFilingDetails': 'filingUrl'})

# display(urls)

urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: '/'.join(url.split('/')[:-1]) + '/Financial_Report.xlsx')

urls.head(10)

,ticker,formType,periodOfReport,filedAt,filingUrl,financialReportsUrl
0,SIF,10-K,2023-09-30,2023-12-29T17:53:23-05:00,https://www.sec.gov/Archives/edgar/data/90168/...,https://www.sec.gov/Archives/edgar/data/90168/...
1,MSBB,10-K,2023-09-30,2023-12-29T17:14:34-05:00,https://www.sec.gov/Archives/edgar/data/196730...,https://www.sec.gov/Archives/edgar/data/196730...
2,WBBA,10-K,2023-07-31,2023-12-29T17:09:26-05:00,https://www.sec.gov/Archives/edgar/data/178741...,https://www.sec.gov/Archives/edgar/data/178741...
3,BDL,10-K,2023-09-30,2023-12-29T16:51:14-05:00,https://www.sec.gov/Archives/edgar/data/12040/...,https://www.sec.gov/Archives/edgar/data/12040/...
4,MRDB,10-K,2023-09-30,2023-12-29T16:46:24-05:00,https://www.sec.gov/Archives/edgar/data/192958...,https://www.sec.gov/Archives/edgar/data/192958...
5,REGX,10-K,2023-09-30,2023-12-29T16:38:07-05:00,https://www.sec.gov/Archives/edgar/data/135968...,https://www.sec.gov/Archives/edgar/data/135968...
6,CTXR,10-K,2023-09-30,2023-12-29T16:05:44-05:00,https://www.sec.gov/Archives/edgar/data/150625...,https://www.sec.gov/Archives/edgar/data/150625...
7,NRT,10-K,2023-10-31,2023-12-29T10:47:42-05:00,https://www.sec.gov/Archives/edgar/data/72633/...,https://www.sec.gov/Archives/edgar/data/72633/...
8,NaN,10-K,2023-09-30,2023-12-29T09:07:12-05:00,https://www.sec.gov/Archives/edgar/data/196709...,https://www.sec.gov/Archives/edgar/data/196709...
9,GTLL,10-K,2023-06-30,2023-12-28T17:55:31-05:00,https://www.sec.gov/Archives/edgar/data/932021...,https://www.sec.gov/Archives/edgar/data/932021...


In [7]:
import requests
single_filing = urls.iloc[4]
print('selected filing: ', single_filing)
financial_report_path = single_filing['financialReportsUrl'].replace('https://www.sec.gov/Archives/edgar/data/', '')

base_url = 'https://archive.sec-api.io/' + financial_report_path

render_api_url = base_url + '?token=' + api_key

response = requests.get(render_api_url)

file_name = f"{single_filing['ticker']}-{single_filing['periodOfReport']}-{single_filing['formType']}.xlsx"
print(file_name)
output = open('./spreadsheets/' + file_name, 'wb')
output.write(response.content)
output.close()


selected filing:  ticker                                                              MRDB
formType                                                            10-K
periodOfReport                                                2023-09-30
filedAt                                        2023-12-29T16:46:24-05:00
filingUrl              https://www.sec.gov/Archives/edgar/data/192958...
financialReportsUrl    https://www.sec.gov/Archives/edgar/data/192958...
Name: 4, dtype: object
MRDB-2023-09-30-10-K.xlsx


In [7]:
# load every sheet into a dataframe
# also, %pip install openpyxl to load the excel sheet
all_sheets_sif = pd.read_excel('spreadsheets/SIF-2023-09-30-10-K.xlsx', sheet_name=None)
all_sheets_mrdb = pd.read_excel('spreadsheets/MRDB-2023-09-30-10-K.xlsx', sheet_name=None)
all_sheets_bdl = pd.read_excel('spreadsheets/BDL-2023-09-30-10-K.xlsx', sheet_name=None)
all_sheets_msbb = pd.read_excel('spreadsheets/MSBB-2023-09-30-10-K.xlsx', sheet_name=None)
all_sheets_wbba = pd.read_excel('spreadsheets/WBBA-2023-07-31-10-K.xlsx', sheet_name=None)

print('SIF', len(all_sheets_sif))
print('MRDB', len(all_sheets_mrdb))
print('BDL', len(all_sheets_bdl))
print('MSSB', len(all_sheets_msbb))
print('WBBA', len(all_sheets_wbba))

#for s in all_sheets.keys():
#    print(s)

sheets = pd.DataFrame({
    'SIF': all_sheets_sif,
    'BDL': all_sheets_bdl,
    'MRDB': all_sheets_mrdb,
    'MSSB': all_sheets_msbb,
    'WBBA': all_sheets_wbba
    })

display(sheets)

SIF 82
MRDB 82
BDL 71
MSSB 70
WBBA 34


,SIF,BDL,MRDB,MSSB,WBBA
Cover Page,Cover Page - ...,NaN,NaN,NaN,NaN
Audit Information,Audit Information 12 Months End...,NaN,Audit Information 12 Months...,NaN,NaN
Consolidated Statements of Oper,Consolidated Statements of Operations - USD...,NaN,Consolidated Statements of Operations and C...,NaN,NaN
Consolidated Statements of Comp,Consolidated Statements of Comprehensive Inc...,NaN,NaN,Consolidated Statements of Comprehensive Inc...,NaN
Consolidated Balance Sheets,Consolidated Balance Sheets - USD ($) $ in ...,NaN,Consolidated Balance Sheets - USD ($) $ in ...,Consolidated Balance Sheets - ...,NaN
...,...,...,...,...,...
Note 9 - Franchise Rights (Deta,NaN,NaN,NaN,NaN,Note 9 - Franchise Rights (Details Narrative...
Note 10 - Accrued Expenses an_2,NaN,NaN,NaN,NaN,Note 10 - Accrued Expenses and Other Payable...
Note 11 - Shareholder Equity (D,NaN,NaN,NaN,NaN,Note 11 - Shareholder Equity (Details Narrat...
Note 12 - General and Adminis_2,NaN,NaN,NaN,NaN,Note 12 - General and Administrative Expense...


In [44]:
# FYI - a loaded excel sheet is a python dict
# a single sheet is a DataFrame
# a single row is a series
income = all_sheets_bdl['CONSOLIDATED STATEMENTS OF INCO']
# display(income)

# print(income.iloc[0].keys()[0])
print(income.iloc[2].to_json())

display(income)


{"CONSOLIDATED STATEMENTS OF INCOME - USD ($) $ in Thousands":"Restaurant food sales","12 Months Ended":107238,"Unnamed: 2":97429}


,CONSOLIDATED STATEMENTS OF INCOME - USD ($) $ in Thousands,12 Months Ended,Unnamed: 2
0,NaN,"Sep. 30, 2023","Oct. 01, 2022"
1,Revenues:,,
2,Restaurant food sales,107238,97429
3,Restaurant bar sales,29000,26198
4,Package store sales,35187,31692
5,Franchise related revenues,1857,1826
6,Rental income,951,814
7,Other operating income,163,173
8,Total,174396,158132
9,Cost of merchandise sold:,,
